In [9]:
# docker pull emacski/tensorflow-serving:latest-linux_arm64
# docker run -t --rm -p 8501:8501 --mount type=bind,source=/tmp/model_name/,target=/models/model_name/ -e MODEL_NAME=model_name emacski/tensorflow-serving:latest-linux_arm64

In [6]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

Digest: sha256:faa03e786c97f07ef34423fccceeec2398ec8a5759259f94d99078f264e9d7af
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 htt

In [7]:
!docker pull tensorflow/serving

Using default tag: latest
latest: Pulling from tensorflow/serving

8a513d66: Pulling fs layer 
b93c73ad: Pulling fs layer 
aef6d08b: Pulling fs layer 
2a459cb8: Pulling fs layer 
Digest: sha256:6c3c199683df6165f5ae28266131722063e9fa012c15065fc4e245ac7d1db980[4A
Status: Downloaded newer image for tensorflow/serving:latest
docker.io/tensorflow/serving:latest


In [8]:
!docker pull tensorflow/serving:latest-gpu

latest-gpu: Pulling from tensorflow/serving

6f815738: Pulling fs layer 
554f7320: Pulling fs layer 
a1b0b7bd: Pulling fs layer 
f85c83c4: Pulling fs layer 
3064088b: Pulling fs layer 
e7a38efd: Pulling fs layer 
c93db753: Pulling fs layer 
72570e59: Pulling fs layer 
0bf766fa: Pulling fs layer 


72570e59: Downloading  215.6MB/463.3MBDownloading  33.44MB/205.9MBDownloading  112.1MB/205.9MBDownloading  57.04MB/2.086GBDownloading  90.96MB/2.086GBDownloading  169.2MB/463.3MB

e7a38efd: Downloading    694MB/2.086GBDownloading  299.2MB/463.3MBDownloading    338MB/463.3MBDownloading    462MB/2.086GBDownloading    560MB/2.086GBDownloading  632.1MB/2.086GB

e7a38efd: Downloading  1.762GB/2.086GBDownloading  1.443GB/2.086GB

Digest: sha256:183724e62d47acc5b9fa93ddbcb7eeedbfb0ead28cbe2a0a6e5fa27abcebbcb6[5ADownloading  1.791GB/2.086GBDownloading  2.032GB/2.086GB
Status: Downloaded newer image for tensorflow/serving:latest-gpu
docker.io/tensorflow/serving:latest-gpu


In [22]:
# conda install tensorflow

In [21]:
import tensorflow as tf

In [23]:
# can also use mnist instead of vgg
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

In [24]:
model = VGG16(weights='imagenet')

553467096/553467096 [==============================] - 110s 0us/step


In [26]:
img_path = 'lion.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

2022-11-22 04:14:13.499851: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


35363/35363 [==============================] - 0s 1us/step
Predicted: [('n04548280', 'wall_clock', 0.31570774), ('n02708093', 'analog_clock', 0.042893667), ('n04116512', 'rubber_eraser', 0.020840228)]


In [27]:
import time
current_time = int(time.time())
path = f"vgg16/{current_time}"
model.save(path)

INFO:tensorflow:Assets written to: vgg16/1669101259/assets


INFO:tensorflow:Assets written to: vgg16/1669101259/assets


In [51]:
path

'vgg16/1669101259'

In [3]:
# # how the folder should look like
# ├── vgg16
# │ ├── 1618397447
# │ │ ├── assets
# │ │ ├── saved_model.pb
# │ │ └── variables

In [40]:
# !sudo docker run -p 8501:8501 --name tfserving_vgg16 --mount type=bind,source=vgg16/,target=vgg16/1669101259/ -e MODEL_NAME=vgg16 -t tensorflow/serving

In [ ]:
!docker pull emacski/tensorflow-serving:latest-linux_arm64

In [36]:
# API endpoint can be accessed at http://localhost:8501. 

In [58]:
import json
import requests

# def run_prediction(data,headers,endpoint):
#     json_response = requests.post(endpoint,data=data,headers=headers)
#     prediction = json.loads(json_response.text)
#     return (decode_predictions(np.array(predictions['predictions']), top=3)[0])

In [57]:
# endpoint = 'http://localhost:8501/v1/models/vgg16:predict'
# data = json.dumps({"instances": x.tolist()})
# headers = {"content-type": "application/json"}
# run_prediction(data,headers,endpoint)

In [45]:
# code from https://cnvrg.io/how-to-serve-a-model-with-tensorflow/
# another way: https://medium.com/analytics-vidhya/deploy-production-ready-keras-model-with-tensorflow-serving-and-docker-85615245a9ed

In [56]:
json_response = requests.post('http://localhost:8501/v1/models/vgg16:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n04548280', 'wall_clock', 0.31570774), ('n02708093', 'analog_clock', 0.042893667), ('n04116512', 'rubber_eraser', 0.020840228)]


In [59]:
# ran docker with
# docker run -t --rm -p 8501:8501 \
#     -v "/Users/csamp/Documents/stack_ab/vgg16/:/models/vgg16/" \
#     -e MODEL_NAME=vgg16 \
#     emacski/tensorflow-serving:latest-linux_arm64

I ran docker with:

`
docker run -t --rm -p 8501:8501 \
    -v "/Users/csamp/Documents/stack_ab/vgg16/:/models/vgg16/" \
    -e MODEL_NAME=vgg16 \
    emacski/tensorflow-serving:latest-linux_arm6`
`